In [1]:
import numpy as np
from matplotlib import pyplot as plt

import sys
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/')

import qml_interface as qmi

In [2]:
# load dE at 20 and 30 Ang for both molecules
dsdg_003664_b20 = np.loadtxt('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_003664/atomic_energies_with_mic.txt')
dsdg_003664_b30 = np.loadtxt('/home/misa/projects/Atomic-Energies/data/ueg_reference/boxsize/dsgdb9nsd_003664_atomic_energies_with_mic_box30.txt')

dsdg_003700_b20 = np.loadtxt('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_003700/atomic_energies_with_mic.txt')
dsdg_003700_b30 = np.loadtxt('/home/misa/projects/Atomic-Energies/data/ueg_reference/boxsize/dsgdb9nsd_003700_atomic_energies_with_mic_box30.txt')


In [66]:
# find the most similar atoms

# make coulomb representations
reps_dsdg_003664 = qmi.generate_atomic_representations([dsdg_003664_b20], [len(dsdg_003664_b20)])
reps_dsdg_003700 = qmi.generate_atomic_representations([dsdg_003700_b20], [len(dsdg_003700_b20)])

# calculate distance matrix

dmatrix = np.zeros((len(reps_dsdg_003664), len(reps_dsdg_003700)))
              
for i, r in enumerate(reps_dsdg_003664):
    for j, r2 in enumerate(reps_dsdg_003700):
        dmatrix[i,j] = np.linalg.norm(r-r2)

# find closest pairs
pairs = []

for i in range(dmatrix.shape[0]):
    dmatrix_row = dmatrix[i]
    ind = np.where( dmatrix_row == np.amin(dmatrix_row[np.where(dmatrix_row > 0)]))
    tmp = [i, ind[0][0]]
    #tmp.sort()
    pairs.append(tmp)
# pairs = []
# import itertools
# pairs.sort()
# pairs = list(pairs for pairs,_ in itertools.groupby(pairs))

In [84]:
pairs

[[0, 0],
 [1, 1],
 [2, 5],
 [3, 6],
 [4, 4],
 [5, 3],
 [6, 2],
 [7, 9],
 [8, 7],
 [9, 8],
 [10, 12],
 [11, 11],
 [12, 10]]

In [75]:
# calculate the energy difference between these atoms at boxsize = 20 and boxsize = 30
def delta_E(comp1, comp2, pairs):
    mu = []
    e = []
    e_at = []
    for p in pairs:
        mu.append(comp1[p[0],4] - comp2[p[1],4])
        e.append(comp1[p[0],5] - comp2[p[1],5])
        e_at.append(comp1[p[0],6] - comp2[p[1],6])
    return(np.array(mu), np.array(e), np.array(e_at))

In [76]:
# boxsize = 20
mu20, e20, e_at20 = delta_E(dsdg_003664_b20, dsdg_003700_b20, pairs)
# boxsize = 30
mu30, e30, e_at30 = delta_E(dsdg_003664_b30, dsdg_003700_b30, pairs)

In [ ]:
# How big is the difference between dE at bs = 20 and bs = 30?

In [77]:
mu20-mu30

array([-7.40952021e-04, -2.59063764e-03, -1.78567115e-03, -6.78603496e-04,
        1.24842316e-03,  4.41679461e-05,  4.03978338e-04,  7.04937649e-03,
       -3.36933954e-03, -7.02622665e-03,  7.73853693e-04,  5.74455066e-03,
        1.49320045e-02])

In [80]:
e20 - e30

array([-0.00444571, -0.01554383, -0.01071403, -0.00542883,  0.00749054,
        0.00030918,  0.00282785,  0.00704938, -0.00336934, -0.00702623,
        0.00077385,  0.00574455,  0.014932  ])

In [79]:
e_at20 - e_at30

array([-0.00387643, -0.01497455, -0.01014475, -0.00485955,  0.00805982,
        0.00087845,  0.00339713,  0.00761865, -0.00280006, -0.00645695,
        0.00134313,  0.00631383,  0.01550128])